# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных

<ul>
<li>children — количество детей в семье;</li>
<li>days_employed — общий трудовой стаж в днях;</li>
<li>dob_years — возраст клиента в годах;</li>
<li>education — уровень образования клиента;</li>
<li>education_id — идентификатор уровня образования;</li>
<li>family_status — семейное положение;</li>
<li>family_status_id — идентификатор семейного положения;</li>
<li>gender — пол клиента;</li>
<li>income_type — тип занятости;</li>
<li>debt — имел ли задолженность по возврату кредитов;</li>
<li>total_income — ежемесячный доход;</li>
<li>purpose — цель получения кредита;</li>
</ul>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Шаг 1. Изучение общей информации о данных

In [2]:
data = pd.read_csv('data.csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**:
Можно заметить, что в таблице разное число non-null объектов в каждом столбце, из этого можно сделать вывод, что в таблице есть пропущенные значения. Также в таблице в столбце days_employed можно увидеть отрицательное количество дней трудового стажа. В столбце education требуется привести все значения к одному регистру.

## Шаг 2. Предобработка данных

### Обработка пропусков

Для того, чтобы получить стаж в годах, возьмем числа столбца days_employed по модулю и поделим на количество дней в году. Трудовой стаж и ежемесячный доход были заполнены в соответствии с типом занятости. Также возьмем количество детей по модулю, чтобы не было отрицательных значений и уберем аномально большие значения в данном столбце (children)

In [5]:
data['days_employed'] = data['days_employed'].abs() / 365
data.loc[data['days_employed'] >= 100, 'days_employed'] = data.loc[data['days_employed'] >= 100, 'days_employed'] / 100
data = data.rename(columns={'days_employed': 'years_employed'})

data['years_employed'] = data.groupby('income_type')['years_employed'].transform(lambda x: x.fillna(x.mean()))
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

data['children'] = data['children'].abs()
data.loc[data['children'] >= 10, 'children'] = data.loc[data['children'] >= 10, 'children'] // 10

display(data.head(10))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,9.322358,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   years_employed    21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


**Вывод**: Были обнаружены значения NaN в столбцах days_employed и total_income. Возможными причинами таких значений могут быть технические неполадки, либо просто незнание человека о своем точном стаже в днях. Для подсчета характерных значений использовалась медиана, так как среднее не всегда отражает действительность, когда некоторые значения сильно выделяются среди большинства (в большую либо меньшею сторону)

### Замена типа данных

Требуется заменить вещественный тип данных на целочисленный. В данной таблице имеется 2 столбца с вещественными значениями: days_employed и total_income

In [7]:
data['years_employed'] = data['years_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   years_employed    21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB
None


### Обработка дубликатов

Приведем строки столбца education к одному регистру

In [8]:
data['education'] = data['education'].str.lower()
display(data.head(10))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Попробуем отыскать полные дубликаты и удалить их

In [9]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   years_employed    21454 non-null  int32 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int32 
 11  purpose           21454 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB
None


**Вывод**: значения столбца education были приведены к одному регистру, а также удалены полные дубликаты, что позволило сократить количество строк до 21454. Дубликаты могут быть связаны как с техническими неполадками, так и с заполнением пользователем данных несколько раз.

### Лемматизация

В ходе лемматизации первым шагом были выделены основные категории целей кредита (свадьба, недвижимость, автомобиль, образование) и объеденены подобные слова в отдельные списки. Далее была создана функция, которая проверяет вхождение значения столбца purpose в определенный список и присваевает ему одну из 4 категорий

In [10]:
new_list = data['purpose'].unique()
 
purpose_vehicle = ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']
purpose_real_estate = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'покупка своего жилья', 'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью']
purpose_wedding = ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']
purpose_education = ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием']

def to_one_purpose(purpose):
    if purpose in purpose_vehicle:
        return 'автомобиль'
    elif purpose in purpose_real_estate:
        return 'недвижимость'
    elif purpose in purpose_wedding:
        return 'свадьба'
    elif purpose in purpose_education:
        return 'образование'

data['purpose'] = data['purpose'].apply(to_one_purpose)
display(data.head(10))

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,9,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


### Категоризация данных

#### Категоризация по количеству детей

In [11]:
children_debt = data[['children', 'debt']]
print(children_debt.groupby('children')['debt'].sum())

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64


#### Категоризация по семейному положению

In [12]:
family_status__debt = data[['family_status', 'debt']]
print(family_status__debt.groupby('family_status')['debt'].sum())

family_status
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
Name: debt, dtype: int64


#### Категоризация по ежемесячному доходу

In [13]:
def income(income):
    if income >= 500000:
        return 'высокий'
    elif income >= 120000:
        return 'средний'
    else:
        return 'низкий'

total_income__debt = data[['total_income', 'debt']]
total_income__debt['total_income'] = total_income__debt['total_income'].apply(income)
print(total_income__debt.groupby('total_income')['debt'].sum())

total_income
высокий      14
низкий      551
средний    1176
Name: debt, dtype: int64


#### Категоризация по цели получения кредита

In [14]:
purpose_debt = data[['purpose', 'debt']]
print(purpose_debt.groupby('purpose')['debt'].sum())

purpose
автомобиль      403
недвижимость    782
образование     370
свадьба         186
Name: debt, dtype: int64


# Шаг 3

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

В ходе категоризации данных построили отдельную таблицу для столбцов children и debt.
children_debt.groupby('children')['debt'].sum() показывает количество тех, у кого есть задолженность
children_debt.groupby('children')['debt'].count() показывает количество всех заемщиков по количеству детей при делении первого на второе и умножении на 100 получим процент тех, у кого есть задолженность в процентах

In [15]:
print(((children_debt.groupby('children')['debt'].sum() / children_debt.groupby('children')['debt'].count())* 100).round(6).astype(str) + '%')

children
0    7.543822%
1    9.165808%
2    9.492481%
3    8.181818%
4    9.756098%
5         0.0%
Name: debt, dtype: object


**Вывод**: количество детей не сильно влияет на возврат кредита в срок, процент невозврата по всем категориям примерно одинаковый. Чуть больше задолженностей у тех, кто имеет 2 детей, больше всего у тех, у кого 4 детей. Исключение составляет только 5 детей, но стоит также учесть, что всего 9 заемщиков с 5 детьми, так что в данном случае отсутствие задолженностей не является показателем.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
print(((family_status__debt.groupby('family_status')['debt'].sum() / family_status__debt.groupby('family_status')['debt'].count()) * 100).round(6).astype(str) + '%')

family_status
Не женат / не замужем     9.75089%
в разводе                7.112971%
вдовец / вдова           6.569343%
гражданский брак         9.347145%
женат / замужем          7.545182%
Name: debt, dtype: object


**Вывод**: здесь можно наблюдать уже больший разброс значений. Меньше всего задолженностей у вдовцов и людей в разводе. Не женатые люди являются главными задолжниками наровне с тем, кто состоит в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
print((total_income__debt.groupby('total_income')['debt'].sum() / total_income__debt.groupby('total_income')['debt'].count() * 100).round(6).astype(str) + '%')

total_income
высокий    6.306306%
низкий     8.049671%
средний    8.174046%
Name: debt, dtype: object


**Вывод**: в некоторой степени уровень дохода влияет на задолженность. Можно заметить, что у людей со средним и низким уровнем дохода примерно одинаковый уровень задолженностей, в то время как у людей с высоким доходом задолженностей на несколько процентов меньше.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
print((purpose_debt.groupby('purpose')['debt'].sum() / purpose_debt.groupby('purpose')['debt'].count() * 100).round(6).astype(str) + '%')

purpose
автомобиль      9.359034%
недвижимость    7.233373%
образование     9.220035%
свадьба         8.003442%
Name: debt, dtype: object


**Вывод**: в ходе процентного соотношения можно сделать вывод, что больше всего задолжников из числа тех, кто берет кредит на автомобиль и образование. Те, кто берут кредит на небольшие или наоборот очень большие суммы возвращают кредит в срок.

## Шаг 4. Общий вывод

Иходные данные были очищены от пропусков, дубликатов, аномальных значений, также были приведены в единому регистру и категоризованы для дальнеших выводов. Было выяснено, что различные факторы стоит учитывать при выдаче кредита: наличие детей, семейное положение, цель взятия кредита и многое другое. Даже пару процентов уже стоит учитывать, выбирая самого надежного заемщика.